# AstroGraphAnomaly — Region Pack Fast (Colab)

Objectif : tester rapidement des “zones” (chunks) tabulaires sans RA/Dec, par exemple :

- `GalaxyCandidates_*.csv.gz`
- `VariSummary_*.csv.gz`

Le runner `tools/run_regions_fast.py` calcule :
- features robustes
- PCA2 embedding
- score “anomaly-like” via distance kNN (z-score + rank01)
- outputs par région + index HTML

Important :
- Sans RA/Dec, pas de sky map ni sphère céleste.
- Si tu veux A→H, il faut enrichir via `gaiadr3.gaia_source` (TAP) sur `source_id`.


In [ ]:
# Setup (works in Colab and in CI checkout)
import os, sys, subprocess
from pathlib import Path

# In CI, the repo is already checked out. In Colab, we clone it.
if not Path("run_workflow.py").exists():
    subprocess.check_call(["git", "clone", "--depth", "1", "https://github.com/dalozedidier-dot/AstroGraphAnomaly.git"])
    os.chdir("AstroGraphAnomaly")

subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "pip"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"])


In [ ]:
# Run GalaxyCandidates (limité)
!rm -rf results/region_pack_fast
!mkdir -p results/region_pack_fast

!python tools/run_regions_fast.py \
  --kind galaxy_candidates \
  --inputs "data/region_pack/raw/GalaxyCandidates_*.csv.gz" \
  --out results/region_pack_fast/galaxy_candidates \
  --max-regions 3 \
  --max-rows 2000


In [ ]:
# Run VariSummary (limité)
!python tools/run_regions_fast.py \
  --kind vari_summary \
  --inputs "data/region_pack/raw/VariSummary_*.csv.gz" \
  --out results/region_pack_fast/vari_summary \
  --max-regions 3 \
  --max-rows 2000


In [ ]:
# Preview dashboards
from pathlib import Path
from IPython.display import HTML, display

for f in [
    Path("results/region_pack_fast/galaxy_candidates/index.html"),
    Path("results/region_pack_fast/vari_summary/index.html"),
]:
    if f.exists():
        print("DASH:", f)
        display(HTML(f.read_text(encoding="utf-8", errors="ignore")[:1_000_000]))
    else:
        print("Missing:", f)
